In [1]:
import re

In [78]:
class Module:
    def __init__(self, name, dest_module_names):
        self.name = name
        self.dest_module_names = dest_module_names
    def sendPulse(self, isHigh):
        for dest_name in self.dest_module_names:
            #Is defined below, I promise ;-)
            pulseQueue.append((self.name, dest_name, isHigh))

In [79]:
class FlipFlop(Module):
    def __init__(self, name, dest_module_names):
        super().__init__(name, dest_module_names)
        self.isOn = False
    def process_pulse(self, isHigh, _):
        if not isHigh:
            if self.isOn:
                self.isOn = False
                self.sendPulse(False)
            else:
                self.isOn = True
                self.sendPulse(True)

In [80]:
class Conjunction(Module):
    def __init__(self, name, dest_module_names):
        super().__init__(name, dest_module_names)
        self.input_model_last_pulse_map = dict() #{name: False for name in input_model_names}
    def process_pulse(self, isHigh, input_model_name):
        self.input_model_last_pulse_map[input_model_name] = isHigh
        if all(self.input_model_last_pulse_map.values()):
            self.sendPulse(False)
        else:
            self.sendPulse(True)

In [140]:
input = "input.txt"
with open(input, 'r') as infile:
    data = [l.strip() for l in infile.readlines()]

In [141]:
module_regex = re.compile('([%&])(\w+) -> (.+)')

In [169]:
module_map = dict()
for module_string in data:
    module_match = re.fullmatch(module_regex, module_string)
    if module_match:
        module_type, module_name, destination_string = module_match.groups()
        if module_type == '%':
            module_map[module_name] = FlipFlop(module_name, destination_string.split(', '))
        else:
            module_map[module_name] = Conjunction(module_name, destination_string.split(', '))
    else:
        #If it doesn't match, it's the broadcaster
        module_name = 'broadcaster'
        destination_string = module_string[15:]
        module_map[module_name] = destination_string.split(', ')
        
        

In [170]:
#Set the incoming modules for each conjunction module.
for module in module_map.values():
    if isinstance(module, Module):
        destlist = module.dest_module_names
    else:
        destlist = module
    for destname in destlist:
        if destname in module_map and isinstance(module_map[destname], Conjunction):
            module_map[destname].input_model_last_pulse_map[module.name] = False

In [171]:
low_pulse_count, high_pulse_count = 0,0
for _ in range(1000):
    #Initialize pulse queue with a low pulse to each broadcaster destination 
    pulseQueue = [('broadcaster', dest, False) for dest in module_map['broadcaster']]
    #The pulse sent to the broadcaster
    low_pulse_count += 1
    while pulseQueue:
        incoming_name, dest_name, isHigh = pulseQueue.pop(0)
        if isHigh:
            high_pulse_count += 1
        else:
            low_pulse_count += 1
        #print(f'{incoming_name} -{"high" if isHigh else "low"}-> {dest_name}')
        #Catch the 'output' module in the second example
        if dest_name in module_map:
            module_map[dest_name].process_pulse(isHigh, incoming_name)

In [173]:
print(high_pulse_count*low_pulse_count)

898731036


## Part 2
rx is targeted by the conjunction module ns, so all of ns' inputs need to be high pulses at the same time. ns' Inputs are: dc, rv, vp, cq. All also conjunction modules. Can I calculate the number of required pushes for each of those to send a high pulse and then calculate the lowest common multiple?

In [192]:
from math import lcm

In [193]:
module_map = dict()
for module_string in data:
    module_match = re.fullmatch(module_regex, module_string)
    if module_match:
        module_type, module_name, destination_string = module_match.groups()
        if module_type == '%':
            module_map[module_name] = FlipFlop(module_name, destination_string.split(', '))
        else:
            module_map[module_name] = Conjunction(module_name, destination_string.split(', '))
    else:
        #If it doesn't match, it's the broadcaster
        module_name = 'broadcaster'
        destination_string = module_string[15:]
        module_map[module_name] = destination_string.split(', ')
        
        

In [194]:
#Set the incoming modules for each conjunction module.
for module in module_map.values():
    if isinstance(module, Module):
        destlist = module.dest_module_names
    else:
        destlist = module
    for destname in destlist:
        if destname in module_map and isinstance(module_map[destname], Conjunction):
            module_map[destname].input_model_last_pulse_map[module.name] = False

In [195]:
button_presses = 0
results = {} #Collect number of presses required for each component 
while True:
    button_presses += 1
    #Initialize pulse queue with a low pulse to each broadcaster destination 
    pulseQueue = [('broadcaster', dest, False) for dest in module_map['broadcaster']]
    while pulseQueue:
        incoming_name, dest_name, isHigh = pulseQueue.pop(0)
        if isHigh:
            for module_of_interest in ['dc', 'rv', 'vp', 'cq']:
                if module_of_interest in results:
                    continue
                if incoming_name == module_of_interest:
                    results[module_of_interest] = button_presses
        #Catch the 'output' module in the second example
        if dest_name in module_map:
            module_map[dest_name].process_pulse(isHigh, incoming_name)
    if len(results) == 4:
        break

In [199]:
lcm(*results.values())

229414480926893

Booyah!